In [21]:
from PIL import Image
from skimage import io

import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

def getImage(train_dir,images,labels,flag):
    for file_name in os.listdir(train_dir):
        file = train_dir + '/' + file_name
        image = Image.open(file).convert('L')
        image = image.resize((16, 48), Image.ANTIALIAS)
        images.append(np.asarray(image))
        labels.append(flag)
        
pos_dir = 'D:\\Anaconda\\projects\\semafor\\true'
neg_dir = 'D:\\Anaconda\\projects\\semafor\\false'
labels = []
images = []
getImage(pos_dir,images,labels,1)
getImage(neg_dir,images,labels,0)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 1.0 / 8, random_state = 42)

In [23]:
from cv2 import HOGDescriptor

def compute_features(image):
    winSize = (16,48)
    blockSize = (16,16)
    blockStride = (8,8)
    cellSize = (8,8)
    nbins = 9
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 100
    nlevels = 32
    hog = HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
    winStride = (8,8)
    padding = (16,16)
    hist = hog.compute(image,winStride,padding)
    return np.reshape(hist, 4500)


features_train = np.array([compute_features(im) for im in X_train])
features_test = np.array([compute_features(im) for im in X_test])

In [29]:
from sklearn import svm
clf = svm.SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, kernel='sigmoid', max_iter=-1, probability=True,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

model = clf.fit(features_train, y_train)

In [30]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, accuracy_score

Y_pred = model.predict(features_test)

print(metrics.classification_report(y_test, Y_pred))
print('Score: %.2f' % accuracy_score(y_test, Y_pred)) 

             precision    recall  f1-score   support

          0       1.00      0.79      0.88        14
          1       0.85      1.00      0.92        17

avg / total       0.92      0.90      0.90        31

Score: 0.90


In [31]:
from sklearn.externals import joblib
joblib.dump(clf, 'model_semafors.pkl') 

['model_semafors.pkl']